In [1]:
import rasterio
import rasterio.warp
import matplotlib.pyplot as plt
import affine
import numpy
import pyproj
import pandas

import starepandas
import pystare

# Load Landsat Granule/Scene

In [2]:
fname = '../tests/data/LC08_L2SP_042036_20210729_20210804_02_T1_SR_B1.TIF'
with rasterio.open(fname) as dataset:
    data = dataset.read()
    mask = dataset.dataset_mask()
    crs = dataset.crs

    # Pixel center instead of pixel corner
    transform = dataset.transform * affine.Affine.translation(0.5, 0.5)

In [3]:
data = data[0, 0:500, 0:500]

In [4]:
indices = numpy.indices(data.shape, dtype='int16')
xy = numpy.array(transform*indices, dtype='int32')

In [5]:
transformer = pyproj.Transformer.from_crs(crs, "epsg:4326")
lat, lon = transformer.transform(xy[0], xy[1])

In [6]:
sids = pystare.from_latlon_2d(lat, lon, adapt_level=True)

In [7]:
landsat = starepandas.STAREDataFrame({'lon': lon.flatten(),
                                      'lat': lat.flatten(),
                                      'band_1': data.flatten()})

In [8]:
landsat = starepandas.STAREDataFrame(landsat, sids=sids.flatten())

# High-Level

In [9]:
# starepandas.read_geotiff(fname, add_xy=True, add_coordinates=True, add_latlon=True, add_sids=False)